<a href="https://colab.research.google.com/github/trituenhantaoio/DeepLearning-Tutorial/blob/master/Autograd_trong_Pytorch_trituenhantao_io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Autograd: Tự động tính vi phân
===================================

Cốt lõi của các mạng nơ ron trong PyTorch là gói ``autograd``.


Gói ``autograd`` cung cấp phép tính vi phân tự động cho mọi phép toán trên Tensor. 

Trước tiên, hãy tìm hiểu một số thuật ngữ cần thiết.

Tensor
--------

``torch.Tensor`` nếu thuộc tính
``.requires_grad`` được đặt bằng ``True``, ``autograd`` sẽ theo dõi mọi phép toán thực hiện trên Tensor đó. Khi đã thực hiện xong, ta có thể gọi phương thức ``.backward()`` và các giá trị đạo hàm được tính một cách tự động. Giá trị này được tính cộng dồn vào thuộc tính ``.grad`` của Tensor.

Để dừng theo dõi một Tensor, ta sử dụng ``.detach()`` để tách nó ra khỏi lịch sử tính toán và các phép toán sau này cũng không được theo dõi.

Ngoài ra chúng ta cũng có thể sử dụng khối ``with torch.no_grad():``. Khối này hữu ích khi chúng ta đánh giá mô hình vì một mô hình có thể chứa các tham số với ``requires_grad=True``, nhưng khi đánh giá thì ta không quan tâm đến đạo hàm.

Một lớp nữa rất quan trọng với ``autograd`` là ``Function`` (hàm).

``Tensor`` và ``Function`` kết nối với nhau thành một đồ thị không có chu trình biểu diễn lịch sử tính toán. Mỗi ``Tensor`` có một thuộc tính ``.grad_fn`` trỏ đến một ``Function`` tạo ra nó (ngoại trừ các Tensor được tạo bởi người dùng, thuộc tính ``grad_fn`` của chúng là ``None``).

Nếu muốn tính đạo hàm, ta có thể gọi ``.backward()`` trên một ``Tensor``. Nếu ``Tensor`` là kiểu số (scalar) (tức là nó chỉ chứa một phần tử), ta không cần chỉ định đối số cho ``backward()``. Nhưng nếu nó có nhiều hơn một phần tử, ta cần phải chỉ định rõ đối số cho ``gradient`` với định dạng phù hợp.



In [ ]:
import torch

Tạo một Tensor và đặt ``requires_grad=True`` để theo dõi các phép toán trên nó



In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Thực hiện phép toán:



In [ ]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


``y`` được tạo từ kết quả của một phép toán nên nó có thuộc tính ``grad_fn``.



In [ ]:
print(y.grad_fn)

Thực hiện thêm phép toán trên ``y``



In [ ]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


``.requires_grad_( ... )`` thay đổi thuộc tính ``requires_grad`` của Tensor hiện tại. Thuộc tính này mặc định là ``False`` nếu không được chỉ định.



In [ ]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


Độ dốc (gradient)
---------
Ta tiến hành lan truyền ngược (backprop)
Vì ``out`` chứa một giá trị scalar, ``out.backward()`` tương đương với ``out.backward(torch.tensor(1.))``.



In [ ]:
out.backward()

In giá trị đạo hàm d(out)/dx




In [ ]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


Kết quả là một ma trận với các số ``4.5``. Gọi *Tensor* ``out``
là “$o$”.
Ta có $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ và $z_i\bigr\rvert_{x_i=1} = 27$.
Do đó,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, và
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



Về mặt toán học, nếu ta có một hàm $\vec{y}=f(\vec{x})$,
thì các giá trị đạo hàm của $\vec{y}$ trên $\vec{x}$
là một ma trận Jacobi:

\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}

Nói chung là, ``torch.autograd`` là công cụ để tính toán trên véc tơ và ma trận Jacobi. Tức là, với một véc tơ
$v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$,
nó sẽ tính được tích $v^{T}\cdot J$. Nếu $v$ là độ dốc của một hàm scalar $l=g\left(\vec{y}\right)$,
tức là,
$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$,
thì theo quy tắc dây chuyền, tích của véc tơ và ma trận Jacobi sẽ là độ dốc của $l$ đối với $\vec{x}$:

\begin{align}J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

(Lưu ý rằng $v^{T}\cdot J$ cho một véc tơ hàng nhưng ta có thể có một véc tơ cột bằng cách lấy $J^{T}\cdot v$.)

Tính chất này của tích véc tơ Jacobi giúp thuận tiện hóa cho việc đưa độ dốc từ ngoài vào trong mô hình với đầu ra không phải dạng số.



Dưới đây là một ví dụ về tích véc tơ Jacobi:



In [ ]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ -498.3744, -1358.6482,   331.9124], grad_fn=<MulBackward0>)


Trong trường hợp này ``y`` không phải scalar. ``torch.autograd`` không thể thực hiện phép toán Jacobi một cách trực tiếp, nhưng nếu chúng ta cần lấy tích véc tơ Jacobi, ta chỉ cần truyền véc tơ vào phương thức dưới dạng đối số của ``backward()`` :



In [ ]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


Ta có thể dừng việc ``autograd`` theo dõi lịch sử tính toán của Tensor có ``requires_grad=True`` bằng cách gói đoạn code trong
``with torch.no_grad():``



In [ ]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


Hoặc sử dụng ``.detach()`` để có được Tensor mới có cùng giá trị nhưng có ``requires_grad=False``:



In [ ]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
